In [46]:
import sys
import json
from pathlib import Path
import sqlalchemy
import pandas as pd
from utils import * 

In [47]:
queries_dir = Path('queries/')
enum_query = read_sql(queries_dir / "1.enum_query.sql")
table_query = read_sql(queries_dir / "3.table_query.sql")

datasets_dir = Path("../datasets/")
haccp_dataset_path = datasets_dir / "haccp_dataset_200619142700.csv"
veg_prdkind_path = datasets_dir / "VegPrdKind_200619145823.csv"
all_prdkind_path = datasets_dir / "AllergyPrdKind_200619145823.csv"
veg_rawmtrl_path = datasets_dir / "VegRawMtrl_200619145823.csv"
all_rawmtrl_path = datasets_dir / "AllergyRawMtrl_200619145823.csv"

In [48]:
if sys.platform.lower() == 'darwin':
    conn = connect(user='postgres', password='0000', db='projectDB', host='localhost', port=5433)
elif sys.platform == 'win32':
    conn = connect(user='postgres', password='1234', db='projectDB', host='localhost', port=5432)

In [49]:
initializ_db(conn)

In [50]:
execute_queries(conn, enum_query)
execute_queries(conn, table_query)

In [51]:
haccp_dataset = pd.read_csv(haccp_dataset_path)
haccp_dataset = haccp_dataset.drop(['rnum', 'productgb', 'prdkindstate'], axis=1)
    
veg_prdkind = pd.read_csv(veg_prdkind_path)
all_prdkind = pd.read_csv(all_prdkind_path)
veg_rawmtrl = pd.read_csv(veg_rawmtrl_path)
all_rawmtrl = pd.read_csv(all_rawmtrl_path)

In [52]:
replace_dict = {
    '(' : ' ',
    ')' : ' ',
    '%' : '%%',
    "'" : '',
    '\n' : ' ',
    '\r' : ' '
}

In [53]:
for data in haccp_dataset.values:
    new_data = []
    for s in data:
        if type(s) == str:
            for org, dst in replace_dict.items():
                s = s.replace(org, dst)
        new_data.append(s)
    reportno, prdnm, rawmtrl, allergy, nutrient, barcode, prdkind, manu, seller, capa, img1, img2 = new_data
    val = "'{reportno}', '{prdnm}', '{rawmtrl}', '{allergy}', '{nutrient}','{barcode}',\
    '{prdkind}', '{manu}', '{seller}', '{capa}', '{img1}', '{img2}'".format(reportno = reportno, prdnm = prdnm, rawmtrl = rawmtrl,
                                                                 allergy = allergy, nutrient = nutrient, barcode = barcode,
                                                                 prdkind = prdkind, manu=manu, seller=seller, capa=capa, img1=img1, img2=img2)
    insert_query = "INSERT INTO foodinfo VALUES({})".format(val)
    execute_query(conn, insert_query, log=False)

In [54]:
for data in veg_prdkind.values:
    new_data = []
    for s in data:
        if type(s) == str:
            for org, dst in replace_dict.items():
                s = s.replace(org, dst)
        new_data.append(s)
    prdKind, vegan, lactoVeg, ovoVeg, lactoOvoVeg, pescoVeg, polloVeg = new_data
    val = "'{}', '{}', '{}', '{}',' {}', '{}', '{}'".format(prdKind, vegan, lactoVeg, ovoVeg, lactoOvoVeg, pescoVeg, polloVeg)
    insert_query = "INSERT INTO VegPrdKind VALUES({})".format(val)
    execute_query(conn, insert_query, log=False)

In [55]:
for data in veg_rawmtrl.values:
    new_data = []
    for s in data:
        if type(s) == str:
            for org, dst in replace_dict.items():
                s = s.replace(org, dst)
        new_data.append(s)
    rawMtrl, vegan, lactoVeg, ovoVeg, lactoOvoVeg, pescoVeg, polloVeg = new_data
    val = "'{}', '{}', '{}', '{}',' {}', '{}', '{}'".format(rawMtrl, vegan, lactoVeg, ovoVeg, lactoOvoVeg, pescoVeg, polloVeg)
    insert_query = "INSERT INTO VegRawMtrl VALUES({})".format(val)
    execute_query(conn, insert_query, log=False)

In [56]:
for data in all_prdkind.values:
    new_data = []
    for s in data:
        if type(s) == str:
            for org, dst in replace_dict.items():
                s = s.replace(org, dst)
        new_data.append(s)
    prdKind, allergy, isCrossReact, parentAllergy, probablility = new_data
    val = "'{}', '{}', '{}', '{}'".format(prdKind, allergy, isCrossReact, parentAllergy, probablility)
    insert_query = "INSERT INTO AllergyPrdKind VALUES({})".format(val)
    execute_query(conn, insert_query, log=False)

In [57]:
for data in all_rawmtrl.values:
    new_data = []
    for s in data:
        if type(s) == str:
            for org, dst in replace_dict.items():
                s = s.replace(org, dst)
        new_data.append(s)
    rawMtrl, allergy, isCrossReact, parentAllergy, probablility = new_data
    val = "'{}', '{}', '{}', '{}', '{}'".format(rawMtrl, allergy, isCrossReact, parentAllergy, probablility)
    insert_query = "INSERT INTO AllergyRawMtrl VALUES({})".format(val)
    execute_query(conn, insert_query)